# Importing the packages

In [5]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.multivariate.manova import MANOVA
import seaborn as sns
import statsmodels.api as sm
from seaborn_qqplot import pplot
import math

# Pre-analysis
## Basic variables

In [6]:
mediapipe1Dir = "../datasets/mediapipe_rectangle.csv"
haar1Dir = "../datasets/haar_rectangle.csv"
mtcnn1Dir = "../datasets/mtcnn_rectangle.csv"

dependant = ["fps", "total_avg_displacement_x_abs", "total_avg_displacement_y_abs"]
plot_vars = dependant

type_dict = {"fps": float, "frames_with_no_detection": float, "area_bb":float, "total_avg_displacement_x":float, "total_avg_displacement_y":float, "total_avg_displacement_x_abs":float, "total_avg_displacement_y_abs":float}

method_dict = {0: "mediapipe", 1: "haar", 2: "mtcnn"}

exp_dict = {0: "rectangular", 1: ""}

alpha = 0.01

## Reading the datasets

In [7]:
mediapipe1 = pd.read_csv(mediapipe1Dir)
mediapipe1["method"] = ["mediapipe" for i in range(0,mediapipe1.shape[0])]
haar1 = pd.read_csv(haar1Dir)
haar1["method"] = ["haar" for i in range(0,haar1.shape[0])]
mtcnn1 = pd.read_csv(mtcnn1Dir)
mtcnn1["method"] = ["mtcnn" for i in range(0,mtcnn1.shape[0])]

experiment1 = pd.concat([mediapipe1, haar1, mtcnn1])

exp_list = [experiment1]

## Take out repeated indexes
for i in range(0, len(exp_list)):
    indexes = [i for i in range(0, exp_list[0].shape[0])]
    exp_list[0].index = indexes

## Asociate adequate type
for i in range(0,len(exp_list)):
    exp_list[i] = exp_list[i].astype(type_dict)
    exp_list[i] = exp_list[i].dropna(0)

/home/pablo/vEnv/ai3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only


# Initial analysis

## Descriptive statistics

In [ ]:
for i in range(0,len(exp_list)):
    print(exp_list[i].describe())

## Scatter matrix

In [ ]:
for i in range(0, len(exp_list)):
    scatter = sns.pairplot(exp_list[i], hue="method")
    scatter.fig.suptitle(f"Scatter plot for experiment {exp_dict[i]}")
    scatter.fig.subplots_adjust(top=0.95)

    scatter.savefig(f"scatter_{exp_dict[i]}.png", dpi=200)

## Normality check
### Shapiro-Wilkinson test

A Shapiro-Wilkinson test is performed to check the normality of the sample obtained

In [ ]:
for i in range(0, len(exp_list)):
    for j in range(0, 3):
        method = exp_list[i].loc[exp_list[i]["method"] == method_dict[j],dependant]
        print(method)
        print(f" - Results of the Shapiro-Wilkinson test for the experiment {exp_dict[i]} with the method {method_dict[0]}: \n            {stats.shapiro(method)}")


### QQ-plot

In [ ]:
for i in range(0, len(exp_list)):
    m1 = exp_list[i].loc[exp_list[i]["method"] == method_dict[0],dependant]
    cov1 = np.cov(m1.T)
    icov1 = np.linalg.inv(cov1)
    left_term1 = np.dot((m1 - np.mean(m1)),icov1)
    total1 = np.dot(left_term1, (m1 - np.mean(m1)).T)
    mahalab1 = np.diag(total1)
    chi1 = np.random.chisquare(9,  m1.shape[0])
    data1 = pd.DataFrame({"mahalab": mahalab1, "chi": chi1} )
    scatt1 = pplot(x="chi", data=data1, y="mahalab")
    scatt1.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[0]}")
    scatt1.savefig(f"qqplot_exp{i}_m1.png", dpi=300)

    m2 = exp_list[i].loc[exp_list[i]["method"] == method_dict[1],dependant]
    cov2 = np.cov(m2.T)
    icov2 = np.linalg.inv(cov2)
    left_term2 = np.dot((m2 - np.mean(m2)),icov2)
    total2 = np.dot(left_term2, (m2 - np.mean(m2)).T)
    mahalab2 = np.diag(total2)
    chi2 = np.random.chisquare(9,  m2.shape[0])
    data2 = pd.DataFrame({"mahalab": mahalab2, "chi": chi2} )
    scatt2 = pplot(x="chi", data=data2, y="mahalab")
    scatt2.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[1]}")
    scatt2.savefig(f"qqplot_exp{i}_m2.png", dpi=300)

    m3 = exp_list[i].loc[exp_list[i]["method"] == method_dict[2],dependant]
    cov3 = np.cov(m3.T)
    icov3 = np.linalg.inv(cov3)
    left_term3 = np.dot((m3 - np.mean(m3)),icov3)
    total3 = np.dot(left_term3, (m3 - np.mean(m3)).T)
    mahalab3 = np.diag(total3)
    chi3 = np.random.chisquare(9,  m3.shape[0])
    data3 = pd.DataFrame({"mahalab": mahalab3, "chi": chi3} )
    scatt3 = pplot(x="chi", data=data3, y="mahalab")
    scatt3.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[2]}")
    scatt3.savefig(f"qqplot_exp{i}_m3.png", dpi=300)

## Tests
### MANOVA
Performing MANOVA test

### Performing a Barttlet test between both populations

#### With a Barttlet test we can check whether the covariances can be considered equal. In case they can be considered equal, a test will be performed to check whether the mean can also be considered equal or not in the case of equal covariances. Elsewhere, the test will be performed for the case of unequal covariances matrices.

In [ ]:
for i in range(0, len(exp_list)):
    ####################################################################################################
    ## Declare the datasets for each method
    m1 = exp_list[i].loc[exp_list[i]["method"] == method_dict[0],dependant]
    m1 = exp_list[i].loc[exp_list[i]["method"] == method_dict[1],dependant]
    m1 = exp_list[i].loc[exp_list[i]["method"] == method_dict[2],dependant]

    ####################################################################################################
    ## Obtain covariance and other relevant variables
    cov1 = np.cov(m1.T)
    cov2 = np.cov(m2.T)
    cov3 = np.cov(m3.T)

    n1 = m1.shape[0]
    n2 = m2.shape[0]
    n3 = m3.shape[0]
    p = cov1.shape[0]

    Sp12 = ((n1 - 1)*cov1 + (n2 - 1)*cov2)/(n1 + n2 - 2)
    Sp13 = ((n1 - 1)*cov1 + (n3 - 1)*cov3)/(n1 + n3 - 2)
    Sp23 = ((n3 - 1)*cov3 + (n2 - 1)*cov2)/(n3 + n2 - 2)

    equalCov12 = False
    equalCov13 = False
    equalCov23 = False

    ####################################################################################################
    ## Barttlet test
    c12 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n1-1)) + (1/(n1-1)) - (1/(n1+n2-1)))
    L12 = c12 * ((n1 + n2 - 2)*np.log(np.linalg.det(Sp12)) - (n1-1)*np.log(np.linalg.det(cov1)) - (n2-1)*np.log(np.linalg.det(cov2)))
    c23 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n2-1)) + (1/(n3-1)) - (1/(n3+n2-1)))
    L23 = c23 * ((n2 + n3 - 2)*np.log(np.linalg.det(Sp23)) - (n2-1)*np.log(np.linalg.det(cov2)) - (n3-1)*np.log(np.linalg.det(cov3)))
    c13 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n1-1)) + (1/(n3-1)) - (1/(n1+n3-1)))
    L13 = c13 * ((n1 + n3 - 2)*np.log(np.linalg.det(Sp13)) - (n1-1)*np.log(np.linalg.det(cov1)) - (n3-1)*np.log(np.linalg.det(cov3)))
    critical = stats.chi2(df=p*(p+1)/2).ppf(alpha)

    ####################################################################################################
    ## Printing the results
    if (L12 > critical):
        print(f"Rejecting H0 when comparing the methods 1 and 2.")
        equalCov12 = True
    else:
        print(f"Accepting H0 when comparing the methods 1 and 2.")

    if (L23 > critical):
        print(f"Rejecting H0 when comparing the methods 2 and 3.")
        equalCov23 = True
    else:
        print(f"Accepting H0 when comparing the methods 2 and 3.")

    if (L13 > critical):
        print(f"Rejecting H0 when comparing the methods 1 and 3.")
        equalCov13 = True
    else:
        print(f"Accepting H0 when comparing the methods 1 and 3.")
    ####################################################################################################
    ## Check for equal mean vector
    T12_sqr = 0
    T13_sqr = 0
    T23_sqr = 0
    f_dist = stats.f

    ############################################################################
    ## Equal Mean 1-2
    if (equalCov12):
        left_term = np.dot((np.mean(m1) - np.mean(m2)).T,np.linalg.inv(((1/n1)+ (1/n2))*Sp12))
        T12 = np.dot(left_term, (np.mean(m1) - np.mean(m2)).T)
        critical = stats.f.ppf(alpha, p, n1 + n2 - p - 1)
        T12 = math.sqrt(T12)
        critical = c12*critical
        if T12 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m1 and m2")
        else:
            print(f"Aceepting H0: means can be considered equal for m1 and m2")
    else:
        left_term = np.dot((np.mean(m1) - np.mean(m2)),np.linalg.inv((cov1/n1)+ (cov2/n2)))
        T12 = np.dot(left_term, (np.mean(m1) - np.mean(m2)).T)
        T12 = (np.mean(m1) - np.mean(m2)) * np.linalg.inv((cov1/n1).T + (cov2/n2).T) * (np.mean(m1) - np.mean(m2)).T
        critical = stats.chi2(df=p*(p+1)/2).ppf(alpha)
        T12 = T12^(1/2)
        critical = c12*critical
        if T12 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m1 and m2")
        else:
            print(f"Aceepting H0: means can be considered equal for m1 and m2")
    
    ############################################################################
    ## Equal Mean 2-3
    if (equalCov23):
        left_term = np.dot((np.mean(m2) - np.mean(m3)).T,np.linalg.inv(((1/n2)+ (1/n3))*Sp23))
        T23 = np.dot(left_term, (np.mean(m2) - np.mean(m3)).T)
        critical = stats.f.ppf(alpha, p, n1 + n3 - p - 1)
        T23 = math.sqrt(T23)
        critical = c23*critical
        if T23 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m2 and m3")
        else:
            print(f"Aceepting H0: means can be considered equal for m2 and m3")
    else:
        left_term = np.dot((np.mean(m2) - np.mean(m3)),np.linalg.inv((cov2/n2)+ (cov3/n3)))
        T23 = np.dot(left_term, (np.mean(m2) - np.mean(m3)).T)
        critical = stats.chi2(df=p*(p+1)/2).ppf(alpha)
        T23 = math.sqrt(T23)
        critical = c23*critical
        if T23 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m2 and m3")
        else:
            print(f"Aceepting H0: means can be considered equal for m2 and m3")

    ############################################################################
    ## Equal Mean 1-3
    if (equalCov13):
        left_term = np.dot((np.mean(m1) - np.mean(m3)).T,np.linalg.inv(((1/n1)+ (1/n3))*Sp13))
        T13 = np.dot(left_term, (np.mean(m1) - np.mean(m3)).T)
        critical = stats.f.ppf(alpha, p, n1 + n3 - p - 1)
        T13 = math.sqrt(T13)
        critical = c13*critical
        if T13 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m1 and m3")
        else:
            print(f"Aceepting H0: means can be considered equal for m1 and m3")
    else:
        left_term = np.dot((np.mean(m1) - np.mean(m3)),np.linalg.inv((cov1/n1)+ (cov3/n3)))
        T13 = np.dot(left_term, (np.mean(m1) - np.mean(m3)).T)
        critical = stats.chi2(df=p*(p+1)/2).ppf(alpha)
        T13 = math.sqrt(T13)
        critical = c13*critical
        if T13 > critical:
            print(f"Rejecting H0: means cannot be considered equal for m1 and m3")
        else:
            print(f"Aceepting H0: means can be considered equal for m1 and m3")
